# RecDP LLM - TextToQA

This component is using pretrained large language model to generate QA dataset for a given text

# Get Started

## 1. Install pyrecdp and dependencies

In [ ]:
! DEBIAN_FRONTEND=noninteractive apt-get install -qq -y openjdk-8-jre
! pip install -q pyrecdp --pre
# ! pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

## 2. Prepare your data

In [ ]:
%mkdir -p /content/test_data
%cd /content/test_data
!wget https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/llm_data/text.parquet

## 3. Generate QA from text

### 3.1 Process with TextToQA

In [ ]:
from pyrecdp.LLM import TextPipeline, ResumableTextPipeline
from pyrecdp.primitives.operations import *

pipeline = TextPipeline(engine_name="spark")
data_path="/content/test_data/text.parquet"
text_key = "text_prompt"
model_name="neural-chat-7b-v3"
out_path = "ResumableTextPipeline_output"
ops = [
    ParquetReader(data_path),
    # TextPrompt(dataset_name="text", prompt_name="generate_qa",new_name=text_key),
    TextToQA(outdir=out_path,model_name=model_name,text_key=text_key),
    ParquetWriter(out_path)
]
pipeline.add_operations(ops)
ret = pipeline.execute()
del pipeline

### 3.2 View processed data

In [1]:
import pandas as pd
data_pd = pd.read_parquet(out_path)
data_pd.head()

,text_prompt,QA_output,Question,Answer
0,You will be provided an article. After reading...,Question: What change in the estimated useful ...,What change in the estimated useful life of c...,"In January 2023, Intel increased the estimate..."
1,You will be provided an article. After reading...,Question: What change in the estimated useful ...,How did this change affect Intel's financial ...,The change is expected to reduce total deprec...
2,You will be provided an article. After reading...,Question: What change in the estimated useful ...,What is the purpose of Intel's Q1 2023 outloo...,Intel estimates a $350 million to $500 millio...
3,You will be provided an article. After reading...,Question: What change in the estimated useful ...,When did Intel hold a public webcast to discu...,Intel held a public webcast at 2 p.m. PST on ...
4,You will be provided an article. After reading...,Question: What change in the estimated useful ...,"Where can the live public webcast, earnings p...","The live public webcast, earnings presentatio..."
